In [1]:
!wget https://raw.githubusercontent.com/ChiThang-50Cent/review-classification-BERT/main/Data/item.json

--2023-12-03 08:31:20--  https://raw.githubusercontent.com/ChiThang-50Cent/review-classification-BERT/main/Data/item.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 627745 (613K) [text/plain]
Saving to: ‘item.json’

item.json           100%[===================>] 613.03K  --.-KB/s    in 0.05s   

2023-12-03 08:31:20 (11.0 MB/s) - ‘item.json’ saved [627745/627745]



In [2]:
!wget https://raw.githubusercontent.com/ChiThang-50Cent/review-classification-BERT/main/Data/test_data.json

--2023-12-03 08:31:20--  https://raw.githubusercontent.com/ChiThang-50Cent/review-classification-BERT/main/Data/test_data.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 128697 (126K) [text/plain]
Saving to: ‘test_data.json’

test_data.json      100%[===================>] 125.68K  --.-KB/s    in 0.03s   

2023-12-03 08:31:21 (4.30 MB/s) - ‘test_data.json’ saved [128697/128697]



In [3]:
!pip install py_vncorenlp

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 19.1 MB/s eta 0:00:00
  Created wheel for py_vncorenlp: filename=py_vncorenlp-0.1.4-py3-none-any.whl size=4307 sha256=f9944f9fcf8f75fd94087d20950ca1b9455c40335a52f21a8124314f7565cd03
  Stored in directory: /root/.cache/pip/wheels/d5/d9/bf/62632cdb007c702a0664091e92a0bb1f18a2fcecbe962d9827
Successfully built py_vncorenlp


In [4]:
import json
import requests
import time
import pandas as pd
import numpy as np

In [5]:
import re
import py_vncorenlp

py_vncorenlp.download_model(save_dir='./')
segment_model = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir='./')

In [6]:
f = open('./test_data.json')
data = json.load(f)
f.close()

In [7]:
data.keys()

dict_keys(['key', 'total', 'no_more', 'items'])

In [8]:
list_items = []

for item in data['items']:
  item_info = {}

  item_info['itemid'] = item['itemid']
  item_info['shopid'] = item['shopid']

  list_items.append(item_info)

In [9]:
list_items[0]

{'itemid': 23320583507, 'shopid': 539031967}

In [10]:
url_to_get_review = 'https://shopee.vn/api/v2/item/get_ratings?exclude_filter=1&filter=0&filter_size=0&flag=1&fold_filter=0&itemid={}&offset=0&relevant_reviews=false&request_source=2&shopid={}&tag_filter=&type={}&variation_filters='

review_rate_data = []

for label in [0, 1]:
  for item in list_items:
    res = requests.get(url_to_get_review.format(item['itemid'],
                                                item['shopid'],
                                                label))
    time.sleep(1)
    list_reviews = res.json()['data']['ratings']

    if list_reviews:
      for review in list_reviews:
        data_ = []

        data_.append(item['itemid'])
        data_.append(item['shopid'])
        data_.append(review['comment'])
        data_.append(review['rating_star'])
        data_.append(label)

        review_rate_data.append(data_)


In [11]:
len(review_rate_data)

206

In [12]:
review_rate_data[2]

[23320583507,
 539031967,
 'Giao hàng nhanh đóng gói cẩn thận, tai nghe đúng như miêu tả nghe ok, còn thời gian nghe được bao lâu chưa test nên chưa biết, giá phù hợp với chất lượng',
 4,
 0]

In [13]:
review_rate_data[1:10]

[[23320583507,
  539031967,
  'Tai nghe đẹp độ hoàn thiện tốt, chất âm nghe hay âm bass chắc shop cho kiểm tra hàng trước khi thanh toán nên khá yên tâm khi đặt \nVề tai nghe chất âm ấm, êm sử dụng liên tục hơn 3h chưa hết pin, nhỏ gọn dùng rất thích\nShop có chính sách bảo hành linh hoạt hơn shop khác là đổi miễn phí 2 chiều ship tận nhà',
  5,
  0],
 [23320583507,
  539031967,
  'Giao hàng nhanh đóng gói cẩn thận, tai nghe đúng như miêu tả nghe ok, còn thời gian nghe được bao lâu chưa test nên chưa biết, giá phù hợp với chất lượng',
  4,
  0],
 [23320583507,
  539031967,
  'Chất lượng sản phẩm:tốt\nTính năng nổi bật:ổn\n\nShop đóng gói kĩ càng và cẩn thận, shipper nhiệt tình và chu đáo, sản phẩm ok, sẽ ủng hộ shop lần sau',
  5,
  0],
 [23320583507,
  539031967,
  'Tính năng nổi bật:Nghe nhạc êm, pin được 4 tiếng\n\nMình xem đánh giá mãi của mọi người mới quyết định mua. Phải nói là âm thanh hay bass đâp tốt lắm\ncảm biến dễ sử dụng và mic đàm thoại tốt ngh

In [14]:
df = pd.DataFrame(review_rate_data, columns=['item_id','shop_id','comment','rate_star','label'])

In [15]:
df['comment'] = df['comment'].replace('', np.nan)

In [ ]:
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173 entries, 0 to 172
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   item_id    173 non-null    int64 
 1   shop_id    173 non-null    int64 
 2   comment    173 non-null    object
 3   rate_star  173 non-null    int64 
 4   label      173 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 6.9+ KB


In [ ]:
df['label'].value_counts()

0    130
1     43
Name: label, dtype: int64

In [ ]:
df['comment'] = df['comment'].apply(lambda x: x.replace('\n', '. '))

df.loc[df['rate_star'] <= 2, 'label'] = 1
df.loc[df['rate_star'] >= 4, 'label'] = 0

In [ ]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030" "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def remove_special_characters(text):
    pattern = r"[^\w\s.,;:?]"
    return re.sub(pattern, "", text)

In [ ]:
df['comment'] = df['comment'].apply(remove_emoji)
df['comment'] = df['comment'].apply(remove_special_characters)

In [ ]:
df['comment'] = df['comment'].apply(lambda x: segment_model.word_segment(x)[0])

In [ ]:
df = df.sort_values(by='rate_star')
df

,item_id,shop_id,comment,rate_star,label
172,15494606385,539031967,Tệ,1,1
148,23320583507,539031967,"quá tệ , k thể kết_nối được , các hình_ảnh fee...",1,1
147,23320583507,539031967,"Nghe kém , bên được bên m , k rõ tiếng , pin s...",1,1
146,23320583507,539031967,Tính_năng nổi_bật : nghe .,1,1
145,23320583507,539031967,"Tai nghe có vấn_đề , liên_hệ shop đổi lại thì ...",1,1
...,...,...,...,...,...
61,23903614372,539031967,Vỏ xinh nhưng chỉ đống đc gần hết thui nha như...,5,0
63,23903614372,539031967,"Mới nghe thấy cũng ok , chắc phải dùng thêm ti...",5,0
64,23903614372,539031967,Vỏ đựng đẹp i12 nghe tạm ổn chất_lượng dùng lâ...,5,0
52,23903614372,539031967,Chất_lượng sản_phẩm : tốt .,5,0


In [ ]:
df.to_csv('./shopee_reviews.csv', index=False)